In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from talus_data_analysis.plot import histogram
from talus_data_analysis.elib import Elib
from talus_data_analysis.load import read_excel_from_gdrive, read_df_from_s3
from talus_data_analysis.save import write_df_to_s3
from dotenv import load_dotenv
import tempfile
import sqlite3
import math
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [ ]:
load_dotenv()

In [ ]:
gauth = GoogleAuth(settings_file="../settings.yaml")

In [ ]:
ENCYCLOPEDIA_BUCKET = "talus-data-pipeline-encyclopedia-bucket"
DATA_FOLDER = "../data/210525_BJ"
S3_FOLDER = "wide/210525_BJ"
PROJECT_NAME = "Dalal Contract"

ELIB_FILE = "RESULTS-quant.elib"
peptide_protein_file = "peptide_proteins_results.csv"
peptide_protein_norm_output = "peptide_proteins_normalized.csv"
msstats_groupcompare_output = "msstats_groupcompare.csv"
comparison_matrix_file = "comparison_matrix.csv"

In [ ]:
peptide_df = read_df_from_s3(bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/RESULTS-quant.elib.peptides.txt", inputformat="txt")
peptide_df = peptide_df.drop("numFragments", axis=1)
# make sure there is one protein per column
peptide_df = peptide_df.drop("Protein", axis=1).join(peptide_df["Protein"].str.split(";", expand=True).stack().reset_index(drop=True, level=1).rename("Protein"))
# melt the dataframe so that each source file is in a seperate row instead of column
peptide_df = peptide_df.melt(id_vars=["Peptide", "Protein"], var_name="SourceFile", value_name="TotalIntensity")
peptide_df["Run"] = peptide_df["SourceFile"].apply(lambda x: x.split(".")[0].split("_")[-1])

In [ ]:
peptide_df

## Template DF

In [ ]:
sample_df = pd.read_excel("~/Downloads/MS_samples_May2021.xlsx")
sample_df = sample_df[(sample_df["Project"] == PROJECT_NAME) & (sample_df["Sample Description"] != "Pooled Sample")]
sample_df["Run"] = sample_df["Run ID"].apply(lambda x: x.split("_")[-1])
sample_df = sample_df.rename(columns={"Sample Name": "BioReplicate", "Sample Description": "Condition"})
sample_df = sample_df[["Run", "BioReplicate", "Condition", "Comparison"]]

In [ ]:
sample_df

In [ ]:
msstats_df = pd.merge(peptide_df, sample_df, how="right", on="Run")

In [ ]:
msstats_df

In [ ]:
## Add a few required columns and rename header to match MSstats convention
msstats_df = msstats_df.drop(["Run", "Comparison"], axis=1)
msstats_df["PrecursorCharge"] = 2
msstats_df["IsotopeLabelType"] = "L"
msstats_df["FragmentIon"] = "y0"
msstats_df["ProductCharge"] = "1"
msstats_df = msstats_df.rename(columns={"Peptide": "PeptideSequence",
                                        "Protein": "ProteinName",
                                        "SourceFile": "Run",
                                        "TotalIntensity": "Intensity"})
msstats_df = msstats_df.drop_duplicates()

In [ ]:
msstats_df = msstats_df.sort_values(by=["PeptideSequence", "Intensity"]).reset_index(drop=True)

In [ ]:
msstats_df

In [ ]:
msstats_df.to_csv(f"{DATA_FOLDER}/{peptide_protein_file}")

In [ ]:
write_df_to_s3(dataframe=msstats_df, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_file.replace('.csv', '.parquet')}", outputformat="parquet")

In [ ]:
def get_comparison_matrix(df, filter_target_func=lambda x:x):
    df = df.sort_values(by="Condition")
    dmso_map = {condition: dmso for (condition, dmso) in zip(df["Condition"], df["Comparison"])}
    
    comp_lol = []
    targets = sorted(dmso_map.keys())
    comp_df_index = []
    for i, s in enumerate(targets):
        comp_list = [0 for t in targets]
        if dmso_map[s] in dmso_map:
            # make dmso 0
            comp_list[targets.index(dmso_map[s])] = -1
            # make target itself 1
            comp_list[i] = 1

            comp_lol.append(comp_list)
            comp_df_index.append(f"{filter_target_func(s)}/{dmso_map[s]}")

    comp_df = pd.DataFrame(comp_lol)
    comp_df.index = comp_df_index
    
    return comp_df

In [ ]:
comp_matrix = get_comparison_matrix(df=sample_df, filter_target_func=lambda x: x.split(" - ")[0])

In [ ]:
comp_matrix

In [ ]:
comp_matrix.to_csv(f"{DATA_FOLDER}/{comparison_matrix_file}")

# Run R Script (MSStats) ...

## Write msstats normalized peptide protein df to s3

In [ ]:
msstats_df_norm = pd.read_csv(f"{DATA_FOLDER}/{peptide_protein_norm_output}")

In [ ]:
msstats_df_norm

In [ ]:
write_df_to_s3(dataframe=msstats_df_norm, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_norm_output.replace('.csv', '.parquet')}", outputformat="parquet")

## Write msstats groupcompare df to s3

In [ ]:
msstats_groupcompare = pd.read_csv(f"{DATA_FOLDER}/{msstats_groupcompare_output}")

In [ ]:
msstats_groupcompare[msstats_groupcompare["adj.pvalue"] > 0]

In [ ]:
write_df_to_s3(dataframe=msstats_groupcompare, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{msstats_groupcompare_output.replace('.csv', '.parquet')}", outputformat="parquet")